<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Dialogue_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec  1 02:52:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers --quiet
!git clone https://github.com/allenai/comet-atomic-2020
!pip install -r ./comet-atomic-2020/requirements.txt --quiet
!wget https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
!unzip comet-atomic_2020_BART.zip

     |████████████████████████████████| 3.1 MB 8.0 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 82.5 MB/s 
     |████████████████████████████████| 895 kB 67.7 MB/s 
     |████████████████████████████████| 3.3 MB 70.9 MB/s 
Cloning into 'comet-atomic-2020'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 166 (delta 42), reused 84 (delta 27), pack-reused 56
Receiving objects: 100% (166/166), 7.15 MiB | 20.75 MiB/s, done.
Resolving deltas: 100% (48/48), done.
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 90 kB 6.0 MB/s 
     |████████████████████████████████| 313 kB 52.8 MB/s 
     |████████████████████████████████| 9.1 MB 60.8 MB/s 
     |████████████████████████████████| 379 kB 89.8 MB/s 
     |████████████████████████████████| 829 kB 75.5 MB/s 
     |████████████████████

In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/scripts_txt"
#data_root = "/content/drive/MyDrive/ANLP21/scripts_sample"
output_dir= "/content/drive/MyDrive/ANLP21/exp"

Mounted at /content/drive


In [4]:
import pandas as pd
import random
import os
import re
import spacy
from collections import Counter
from joblib import Parallel, delayed
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [5]:
import sys
sys.path.insert(1,r'./comet-atomic-2020/models/comet_atomic2020_bart')
from generation_example import Comet
print("model loading ...")
comet = Comet("comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")

model loading ...
model loaded


In [6]:
relations = ["CapableOf",  "Desires",  "MotivatedByGoal",  "xAttr", "xNeed", "xReact", "xReason", "xWant"]

vocabulary = set()

film_bags = {}

def register_in_bag(film, token, relation):
    if film not in film_bags:
      film_bags[film] = {}
    if(relation not in film_bags[film]):
      film_bags[film][relation] = set()
    film_bags[film][relation].add(token)
    vocabulary.add(token)
    
def process_results(film, results, relations):
    for i in range(len(relations)):
      relation = relations[i]
      for result in results[i]:
        tokens = result.split(' ')
        for token in tokens:
          if token not in nlp.Defaults.stop_words:
            register_in_bag(film, token, relation)

log = []
def infer(film, item, relations):
    queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
    results = comet.generate(queries, decode_method="beam", num_generate=5)
    if film not in log:
      print(film, end=",")
      log.append(film)
    process_results(film, results, relations)

In [7]:
df1 = pd.read_csv("/content/drive/MyDrive/ANLP21/data.csv")

In [8]:
# df1.apply(lambda row: infer(row['film'], row['item'], relations), axis=1)

In [ ]:
import json
import matplotlib.pyplot as plt
%matplotlib inline

def infer(item):
  queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
  results = comet.generate(queries, decode_method="beam", num_generate=5)
  return results

film_group = df1.groupby('film')
df_concat = []
film_names = []
for film, group in film_group:
  print(film)
  #The following three ways were quicker ways but they don't work for some reason
  # if you want to work with these make sure to convert the results array in infer into json or string before returning
  # group['inference'] = comet.generate(["{} {} [GEN]".format(group['item'], relation) for relation in relations], decode_method="beam", num_generate=5)
  # group['inference'] = group['item'].apply((lambda x: infer(x)), axis=1)
  # group['inference'] = infer(group['item'])
  # for i in range(len(relations)):
  #   # group[str(relations[i])] = group['inference'].apply((lambda x: parse_inf(x, i)), axis=1)
  #   group[str(relations[i])] = parse_inf(group['inference'], i)
  for i in range(len(relations)):
    group[relations[i]] = ["" for i in range(len(group))]

  rel_inf = []
  for ind in group.index:
    item = group['item'][ind]
    results = infer(item)
    for i in range(len(relations)):
      group[relations[i]][ind] = results[i]
  
  df_concat.append(group)
  film_names.append(film)
  group.to_csv(film+'.csv')
  files.download(film+'.csv')

10_things_i_hate_about_you.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df = pd.concat(df_concat)
df.to_csv('concatenated.csv')
files.download('concatenated.csv')
df

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud

def word_cloud(tokens, film_name, relation):
  wordcloud = WordCloud(width = 800, height = 800,
                  background_color ='white',
                  stopwords = nlp.Defaults.stop_words,
                  min_font_size = 10).generate(tokens)
  
  # plot the WordCloud image                      
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  
  plt.show()
  plt.savefig(film_name+'_'+relation+'.png')
  files.download(film_name+'_'+relation+'.png')

for d in range(len(df_concat)):
    group = df_concat[d]
    film_name = film_names[d]
    for i in range(len(relations)):
      tokens = set()
      for l in group[relations[i]]:
        for word in list(l):
          tokens.add(word)
      tokens = str(tokens)
      print(film_name, relations[i])
      word_cloud(tokens, film_name, relations[i])



In [ ]:
for d in range(len(df_concat)):
  group = df_concat[d]
  film_name = film_names[d]
  for i in range(len(relations)):
    files.download(film_name+'_'+relations[i]+'.png')
